Das Jupyter Notebook muss mit Python >= 3.10 ausgeführt werden (getestet in 3.12)

In [1]:
!pip install gymnasium
!pip install numpy
!pip install pygame # todo braucht man pygame wirklich?


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\Kai\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\Kai\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\Kai\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [2]:
!pip install torch


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\Kai\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [7]:
import torch

print(torch.__version__)

2.5.1+cpu


In [2]:
!git clone https://github.com/lychanl/Gymnasium_Snake_Game

Cloning into 'Gymnasium_Snake_Game'...


In [3]:
import sys

sys.path.append('./Gymnasium_Snake_Game') # python erkennt das neu angelegte Verzeichnis manchmal nicht, deswegen hier explixit laden

Der Code ist von https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html

In [9]:
import random
from collections import namedtuple, deque

Transition = namedtuple('Transition', ('state', 'action', 'next_state', 'reward'))

class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [10]:
import torch.nn as nn
import torch.nn.functional as F

class DQN(nn.Module):

    def __init__(self, n_observations, n_actions):
        super(DQN, self).__init__()
        print(f"n_observations: {n_observations}")  # Debugging
        self.layer1 = nn.Linear(n_observations, 128)
        self.layer2 = nn.Linear(128, 128)
        self.layer3 = nn.Linear(128, n_actions)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = x.view(x.size(0), -1)  # Input flatten (batch size, höhe,breite,rgb)
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.layer3(x)

In [11]:
import gymnasium_snake_game
import gymnasium
import numpy as np
import math
import matplotlib
import matplotlib.pyplot as plt

from IPython import display

import torch
import torch.optim as optim

# BATCH_SIZE is the number of transitions sampled from the replay buffer
# GAMMA is the discount factor as mentioned in the previous section
# EPS_START is the starting value of epsilon
# EPS_END is the final value of epsilon
# EPS_DECAY controls the rate of exponential decay of epsilon, higher means a slower decay
# TAU is the update rate of the target network
# LR is the learning rate of the ``AdamW`` optimizer
BATCH_SIZE = 128
GAMMA = 0.99
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 1000
TAU = 0.005
LR = 1e-4

# Falls das Jupyter Notebook 
device = torch.device(
    "cuda" if torch.cuda.is_available() else
    "mps" if torch.backends.mps.is_available() else
    "cpu"
)


options = {
    'fps': 30,
    'max_step': 500,
    'init_length': 4,
    'food_reward': 2.0,
    'dist_reward': 0.1, # todo ist das ein sinnvoller dist reward?
    'living_bonus': 0.0,
    'death_penalty': -1.0,
    'width': 20,
    'height': 20,
}

env = gymnasium.make('Snake-v1', render_mode='human', **options)


# env.reset() # braucht man nicht!?


# Get number of actions from gym action space
n_actions = env.action_space.n
# Get the number of state observations
state, info = env.reset()
n_observations = len(state)

print(n_observations)

# n_observations = 40 * 40 * 3 # höhe, breite, rgb farben
print(state.shape)
n_observations = state.shape[0] * state.shape[1] * state.shape[2]


policy_net = DQN(n_observations, n_actions).to(device)
target_net = DQN(n_observations, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())

optimizer = optim.AdamW(policy_net.parameters(), lr=LR, amsgrad=True)
memory = ReplayMemory(10000)


steps_done = 0


def select_action(state):
    global steps_done
    sample = random.random()
    
    eps_threshold = EPS_END + (EPS_START - EPS_END) * math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return the largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            # return policy_net(state).max(1).indices.view(1, 1)
            return policy_net(state).argmax(dim=1).view(1, 1)
    else:
        # return torch.tensor([[env.action_space.sample()]], device=device, dtype=torch.long)
        return torch.tensor([[random.randint(0, 3)]], device=device, dtype=torch.long)

episode_durations = []


def plot_durations(show_result=False):
    plt.figure(1)
    durations_t = torch.tensor(episode_durations, dtype=torch.float)
    if show_result:
        plt.title('Result')
    else:
        plt.clf()
        plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(durations_t.numpy())
    # Take 100 episode averages and plot them too
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # pause a bit so that plots are updated
    display.display(plt.gcf())
    display.clear_output(wait=True)

20
(20, 20, 3)
n_observations: 1200
n_observations: 1200


In [12]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1).values
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    with torch.no_grad():
        next_state_values[non_final_mask] = target_net(non_final_next_states).max(1).values
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    # In-place gradient clipping
    torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)
    optimizer.step()

In [13]:
from itertools import count

if torch.cuda.is_available() or torch.backends.mps.is_available():
    num_episodes = 600
else:
    num_episodes = 50

for i_episode in range(num_episodes):
    # Initialize the environment and get its state
    state, info = env.reset()
    # print(f"state shape: {state.shape}")
    state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
    # print(f"state shape: {state.shape}")
    # state = torch.tensor(state, dtype=torch.float32, device=device).flatten().unsqueeze(0)
    # print(f"state shape: {state_fl.shape}")
    for t in count():
        action = select_action(state)
        observation, reward, terminated, truncated, _ = env.step(action.item())
        reward = torch.tensor([reward], device=device)
        done = terminated or truncated

        if terminated:
            next_state = None
        else:
            next_state = torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0)

        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the policy network)
        optimize_model()

        # Soft update of the target network's weights
        # θ′ ← τ θ + (1 −τ )θ′
        target_net_state_dict = target_net.state_dict()
        policy_net_state_dict = policy_net.state_dict()
        for key in policy_net_state_dict:
            target_net_state_dict[key] = policy_net_state_dict[key]*TAU + target_net_state_dict[key]*(1-TAU)
        target_net.load_state_dict(target_net_state_dict)

        if done:
            episode_durations.append(t + 1)
            plot_durations()
            break

print('Complete')
plot_durations(show_result=True)
plt.ioff()
plt.show()

<Figure size 640x480 with 0 Axes>

In [2]:
import gymnasium_snake_game
import gymnasium
import numpy as np

# Aus der Readme von: https://github.com/lychanl/Gymnasium_Snake_Game
options = {
    'fps': 30,
    'max_step': 500,
    'init_length': 4,
    'food_reward': 2.0,
    'dist_reward': 0.1, # todo ist das ein sinnvoller dist reward?
    'living_bonus': 0.0,
    'death_penalty': -1.0,
    'width': 40,
    'height': 40,
}

env = gymnasium.make('Snake-v1', render_mode='human', **options)

states = env.observation_space.shape[0]
actions = env.action_space.n
print(actions) # Es gibt 4 actions (0-3), die die Snake ausführen kann



# env.reset()

# # for human playing
# env.unwrapped.play()
# 
# # for ai playing
# while True:
#     obs, reward, done, truncated, info = env.step(env.action_space.sample())
#     if done:
#         break
env.close()

ImportError: cannot import name 'model_from_config' from 'tensorflow.keras.models' (C:\Users\Kai\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\_tf_keras\keras\models\__init__.py)

In [1]:
!pip install torch torchvision 

   ---------------------------------------- 0.0/203.0 MB ? eta -:--:--
   ---------------------------------------- 0.1/203.0 MB 2.0 MB/s eta 0:01:43
   ---------------------------------------- 0.4/203.0 MB 5.0 MB/s eta 0:00:41
   ---------------------------------------- 1.4/203.0 MB 9.9 MB/s eta 0:00:21
    --------------------------------------- 2.9/203.0 MB 15.5 MB/s eta 0:00:13
    --------------------------------------- 4.1/203.0 MB 18.8 MB/s eta 0:00:11
    --------------------------------------- 4.1/203.0 MB 18.8 MB/s eta 0:00:11
   - -------------------------------------- 6.1/203.0 MB 18.5 MB/s eta 0:00:11
   - -------------------------------------- 8.2/203.0 MB 21.9 MB/s eta 0:00:09
   - -------------------------------------- 9.0/203.0 MB 21.2 MB/s eta 0:00:10
   -- ------------------------------------- 12.0/203.0 MB 32.7 MB/s eta 0:00:06
   -- ------------------------------------- 13.6/203.0 MB 32.7 MB/s eta 0:00:06
   -- ------------------------------------- 15.1/203.0 MB 38.


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\Kai\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip
